In [1]:
import multiprocessing
print(f"Available CPU cores: {multiprocessing.cpu_count()}")

Available CPU cores: 10


In [2]:
# Install psutil if you haven't
# !pip install psutil

import psutil

def get_memory_info():
    # Get memory information
    mem = psutil.virtual_memory()

    # Convert to GB for better readability
    total_gb = mem.total / (1024**3)
    available_gb = mem.available / (1024**3)
    used_gb = mem.used / (1024**3)

    print(f"Memory Status:")
    print(f"Total: {total_gb:.1f}GB")
    print(f"Available: {available_gb:.1f}GB")
    print(f"Used: {used_gb:.1f}GB")

    # Calculate recommended Spark memory (70% of available memory)
    recommended_spark = (mem.available * 0.7) / (1024**3)
    print(f"\nRecommended Spark driver memory: {recommended_spark:.1f}GB")

    return recommended_spark

# Get recommended memory
recommended_gb = get_memory_info()
recommended_gb

Memory Status:
Total: 64.0GB
Available: 19.8GB
Used: 23.3GB

Recommended Spark driver memory: 13.9GB


13.879110717773436

In [3]:
import pyspark
from pyspark.sql import SparkSession

In [4]:
import os
os.environ["JAVA_HOME"] = "/Users/schwinger42/miniforge3/envs/de-zoomcamp/lib/jvm"
!echo $JAVA_HOME
!$JAVA_HOME/bin/java -version

/Users/schwinger42/miniforge3/envs/de-zoomcamp/lib/jvm
openjdk version "23.0.2" 2025-01-21
OpenJDK Runtime Environment Zulu23.32+11-CA (build 23.0.2+7)
OpenJDK 64-Bit Server VM Zulu23.32+11-CA (build 23.0.2+7, mixed mode, sharing)


In [30]:
# 1. 首先檢查 Java 版本
import subprocess

def check_java_version():
    try:
        result = subprocess.check_output(['java', '-version'], stderr=subprocess.STDOUT).decode()
        print("當前 Java 版本信息：")
        print(result)
        return result
    except:
        print("無法獲取 Java 版本信息")
        return None
check_java_version()

當前 Java 版本信息：
openjdk version "23.0.2" 2025-01-21
OpenJDK Runtime Environment Zulu23.32+11-CA (build 23.0.2+7)
OpenJDK 64-Bit Server VM Zulu23.32+11-CA (build 23.0.2+7, mixed mode, sharing)



'openjdk version "23.0.2" 2025-01-21\nOpenJDK Runtime Environment Zulu23.32+11-CA (build 23.0.2+7)\nOpenJDK 64-Bit Server VM Zulu23.32+11-CA (build 23.0.2+7, mixed mode, sharing)\n'

In [6]:
active_session = SparkSession.getActiveSession()
if active_session:
    active_session.stop()

spark = SparkSession.builder \
    .config("spark.driver.extraJavaOptions", "-Djava.security.manager=allow") \
    .config("spark.hadoop.hadoop.security.authentication", "simple") \
    .config("spark.hadoop.hadoop.security.authorization", "false") \
    .config("spark.driver.memory", "10g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "2g") \
    .config("spark.sql.shuffle.partitions", "20") \
    .config("spark.eventLog.gcMetrics.youngGenerationGarbageCollectors", "G1 Young Generation") \
    .config("spark.eventLog.gcMetrics.oldGenerationGarbageCollectors", "G1 Old Generation,G1 Concurrent GC") \
    .master("local[*]") \
    .appName("Course 531") \
    .getOrCreate()


25/03/07 01:26:01 WARN Utils: Your hostname, BarondeMacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.68.58 instead (on interface en0)
25/03/07 01:26:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/07 01:26:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
# !wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-01.parquet

In [8]:
!wc -l fhvhv_tripdata_2024-01.parquet

 1899765 fhvhv_tripdata_2024-01.parquet


In [9]:
df = spark.read.parquet("fhvhv_tripdata_2024-01.parquet")

In [29]:
df.show()

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [11]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B03404', originating_base_num='B03404', request_datetime=datetime.datetime(2024, 1, 1, 0, 21, 47), on_scene_datetime=datetime.datetime(2024, 1, 1, 0, 25, 6), pickup_datetime=datetime.datetime(2024, 1, 1, 0, 28, 8), dropoff_datetime=datetime.datetime(2024, 1, 1, 1, 5, 39), PULocationID=161, DOLocationID=158, trip_miles=2.83, trip_time=2251, base_passenger_fare=45.61, tolls=0.0, bcf=1.25, sales_tax=4.05, congestion_surcharge=2.75, airport_fee=0.0, tips=0.0, driver_pay=40.18, shared_request_flag='N', shared_match_flag='N', access_a_ride_flag='N', wav_request_flag='N', wav_match_flag='N'),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B03404', originating_base_num='B03404', request_datetime=datetime.datetime(2024, 1, 1, 0, 10, 56), on_scene_datetime=datetime.datetime(2024, 1, 1, 0, 11, 8), pickup_datetime=datetime.datetime(2024, 1, 1, 0, 12, 53), dropoff_datetime=datetime.datetime(2024, 1, 1, 0, 20, 5), PULocationID=137, DOLoc

In [12]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampNTZType(), True), StructField('on_scene_datetime', TimestampNTZType(), True), StructField('pickup_datetime', TimestampNTZType(), True), StructField('dropoff_datetime', TimestampNTZType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', LongType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('sha

In [13]:
import pandas as pd

In [14]:
# !pip install --upgrade pandas pyarrow
# !pip install fastparquet
# turn head 100 rows into head.parquet
df.limit(100).write.parquet("head.parquet", mode="overwrite")
df_pd = pd.read_parquet("head.parquet", engine="fastparquet")

In [15]:
df_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   hvfhs_license_num     100 non-null    object        
 1   dispatching_base_num  100 non-null    object        
 2   originating_base_num  71 non-null     object        
 3   request_datetime      100 non-null    datetime64[us]
 4   on_scene_datetime     71 non-null     datetime64[us]
 5   pickup_datetime       100 non-null    datetime64[us]
 6   dropoff_datetime      100 non-null    datetime64[us]
 7   PULocationID          100 non-null    int32         
 8   DOLocationID          100 non-null    int32         
 9   trip_miles            100 non-null    float64       
 10  trip_time             100 non-null    int64         
 11  base_passenger_fare   100 non-null    float64       
 12  tolls                 100 non-null    float64       
 13  bcf                  

In [16]:
df_pd.dtypes

hvfhs_license_num               object
dispatching_base_num            object
originating_base_num            object
request_datetime        datetime64[us]
on_scene_datetime       datetime64[us]
pickup_datetime         datetime64[us]
dropoff_datetime        datetime64[us]
PULocationID                     int32
DOLocationID                     int32
trip_miles                     float64
trip_time                        int64
base_passenger_fare            float64
tolls                          float64
bcf                            float64
sales_tax                      float64
congestion_surcharge           float64
airport_fee                    float64
tips                           float64
driver_pay                     float64
shared_request_flag             object
shared_match_flag               object
access_a_ride_flag              object
wav_request_flag                object
wav_match_flag                  object
dtype: object

In [17]:
# spark.createDataFrame(df_pd).schema

In [18]:
def calculate_optimal_partitions(df, target_size_mb=128):
    total_size_bytes = df.count() * len(df.schema.fields) # Approximate size
    total_size_mb = total_size_bytes / (1024 * 1024)
    return max(20, int(total_size_mb / target_size_mb))

num_partitions = calculate_optimal_partitions(df)
num_partitions

20

In [19]:
df = df.repartition(20)

In [20]:
# 先計算並顯示每個分區的數據分佈
from pyspark.sql.functions import spark_partition_id

# 檢查分區數據分佈
df.groupBy(spark_partition_id()).count().orderBy('count', ascending=False).show()

# 寫入時添加壓縮選項
df = df.repartition(20)
df.write \
    .option("compression", "snappy") \
    .mode("overwrite") \
    .parquet("fhvhv/2024/01")


+--------------------+------+
|SPARK_PARTITION_ID()| count|
+--------------------+------+
|                   9|983198|
|                  11|983198|
|                  12|983198|
|                  14|983198|
|                  15|983198|
|                  16|983198|
|                  17|983198|
|                  10|983197|
|                  13|983197|
|                  18|983197|
|                  19|983197|
|                   0|983196|
|                   6|983196|
|                   1|983195|
|                   2|983195|
|                   4|983195|
|                   5|983195|
|                   7|983195|
|                   8|983195|
|                   3|983194|
+--------------------+------+



In [31]:
# Simple test to check if Spark session is alive
try:
    spark.range(1).count()
    print("Spark session is still active")
except Exception as e:
    print("Spark session needs to be restarted:", str(e))


Spark session is still active


In [22]:
# # Get all current configurations
# current_conf = spark.sparkContext.getConf().getAll()
# for item in current_conf:
#     print(item)
#
# # For specific memory-related configs
# print("\nKey Memory Configurations:")
# memory_configs = [
#     "spark.driver.memory",
#     "spark.executor.memory",
#     "spark.memory.offHeap.enabled",
#     "spark.memory.offHeap.size"
# ]
# for config in memory_configs:
#     print(f"{config}: {spark.conf.get(config, 'Not Set')}")
#
# # Get current number of partitions
# print(f"\nCurrent number of partitions: {df.rdd.getNumPartitions()}")


('spark.sql.shuffle.partitions', '20')
('spark.executor.memory', '2g')
('spark.app.id', 'local-1741281962358')
('spark.hadoop.hadoop.security.authentication', 'simple')
('spark.driver.memory', '10g')
('spark.memory.offHeap.size', '2g')
('spark.executor.id', 'driver')
('spark.driver.host', '192.168.68.58')
('spark.memory.offHeap.enabled', 'true')
('spark.driver.port', '49663')
('spark.eventLog.gcMetrics.oldGenerationGarbageCollectors', 'G1 Old Generation,G1 Concurrent GC')
('spark.driver.extraJavaOptions', '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.a


Current number of partitions: 20


In [23]:
import subprocess

def check_gc_info():
    try:
        # 使用 jinfo 命令查看 JVM 參數
        java_pid = subprocess.check_output(['jps']).decode()
        print("Java 進程信息：")
        print(java_pid)

        # 使用 java -XX:+PrintFlagsFinal 來查看 GC 相關配置
        gc_flags = subprocess.check_output(['java', '-XX:+PrintFlagsFinal', '-version'],
                                           stderr=subprocess.STDOUT).decode()
        print("\nGC 相關配置：")
        # 過濾出 GC 相關的配置
        for line in gc_flags.split('\n'):
            if 'GC' in line:
                print(line)

    except subprocess.CalledProcessError as e:
        print("執行命令失敗:", e.output.decode())
    except Exception as e:
        print("發生錯誤:", str(e))

# 執行檢查
check_gc_info()


Java 進程信息：
21701 Jps
21557 SparkSubmit
92731 Main
38045 
21021 RemoteJdbcServer


GC 相關配置：
    uintx AdaptiveSizeMajorGCDecayTimeScale        = 10                                        {product} {default}
     uint ConcGCThreads                            = 2                                         {product} {ergonomic}
     bool DisableExplicitGC                        = false                                     {product} {default}
     bool ExplicitGCInvokesConcurrent              = false                                     {product} {default}
     uint FullGCHeapDumpLimit                      = 0                                      {manageable} {default}
    uintx G1MixedGCCountTarget                     = 8                                         {product} {default}
    uintx G1PeriodicGCInterval                     = 0                                      {manageable} {default}
     bool G1PeriodicGCInvokesConcurrent            = true                                      {produc

In [24]:
df_read = spark.read.parquet("fhvhv/2024/01")

In [26]:
df_read.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp_ntz (nullable = true)
 |-- on_scene_datetime: timestamp_ntz (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropoff_datetime: timestamp_ntz (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_

We can observe that `pickup_datetime`'s data type is `timestamp_ntz` which means it is a timestamp without time zone. We can convert it to a `timestamp` data type which includes time zone information. We can also convert `dropoff_datetime` to a `timestamp` data type.

In [28]:
# selcting columns 'pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID' where 'hvfhs_license_num' is 'HV0003'
df_read.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID').filter(df_read.hvfhs_license_num == 'HV0003').show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2024-01-03 19:27:04|2024-01-03 19:35:24|          90|         114|
|2024-01-02 22:36:17|2024-01-02 22:56:33|         138|          79|
|2024-01-02 00:24:09|2024-01-02 00:52:53|         132|         234|
|2024-01-01 00:21:12|2024-01-01 00:42:30|         148|         255|
|2024-01-03 09:07:34|2024-01-03 09:13:22|         176|         172|
|2024-01-03 16:41:02|2024-01-03 16:48:31|          26|          26|
|2024-01-04 08:29:06|2024-01-04 08:33:12|         200|         200|
|2024-01-02 09:14:01|2024-01-02 09:23:37|          95|         196|
|2024-01-04 18:38:42|2024-01-04 18:45:09|         119|          69|
|2024-01-02 17:42:43|2024-01-02 17:48:51|         242|         182|
|2024-01-02 22:20:16|2024-01-02 22:36:59|         232|          49|
|2024-01-04 10:17:10|2024-01-04 11:19:30|       

In [32]:
from pyspark.sql import functions as F
df_read \
    .withColumn('pickup_date', F.to_date('pickup_datetime')) \
    .withColumn('dropoff_date', F.to_date('dropoff_datetime')) \
    .select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .filter(df_read.hvfhs_license_num == 'HV0003') \
    .show()

+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2024-01-03|  2024-01-03|          90|         114|
| 2024-01-02|  2024-01-02|         138|          79|
| 2024-01-02|  2024-01-02|         132|         234|
| 2024-01-01|  2024-01-01|         148|         255|
| 2024-01-03|  2024-01-03|         176|         172|
| 2024-01-03|  2024-01-03|          26|          26|
| 2024-01-04|  2024-01-04|         200|         200|
| 2024-01-02|  2024-01-02|          95|         196|
| 2024-01-04|  2024-01-04|         119|          69|
| 2024-01-02|  2024-01-02|         242|         182|
| 2024-01-02|  2024-01-02|         232|          49|
| 2024-01-04|  2024-01-04|         145|         265|
| 2024-01-01|  2024-01-01|         164|         231|
| 2024-01-02|  2024-01-02|          95|          92|
| 2024-01-04|  2024-01-04|          90|         107|
| 2024-01-02|  2024-01-02|          86|       

In [33]:
# create a user defined function that SQL can't do
from pyspark.sql.types import StringType

def get_trip_duration(pickup, dropoff):
    return str(dropoff - pickup)

trip_duration_udf = F.udf(get_trip_duration, StringType())

df_read \
    .withColumn('trip_duration', trip_duration_udf('pickup_datetime', 'dropoff_datetime')) \
    .select('trip_duration') \
    .show()

+-------------+
|trip_duration|
+-------------+
|      0:08:20|
|      0:20:16|
|      0:19:16|
|      0:28:44|
|      0:30:18|
|      0:21:18|
|      0:05:48|
|      0:07:29|
|      0:04:06|
|      0:26:15|
|      0:09:36|
|      0:06:27|
|      0:06:08|
|      0:14:04|
|      0:08:57|
|      0:21:01|
|      0:16:43|
|      0:03:13|
|      1:02:20|
|      0:20:07|
+-------------+
only showing top 20 rows

